In [87]:
import pandas as pd
import sqlite3 as s3
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

go_sales = s3.connect('go_sales.sqlite')

order_details = pd.read_sql_query('SELECT * FROM order_details', go_sales)
order_header = pd.read_sql_query('SELECT * FROM order_header', go_sales)
returned_item = pd.read_sql_query('SELECT * FROM returned_item', go_sales)

In [97]:
merged_orders = pd.merge(order_details, order_header, left_on='ORDER_NUMBER', how='inner', right_on='ORDER_NUMBER')
returned_orders = pd.merge(merged_orders, returned_item, left_on='ORDER_DETAIL_CODE', how='outer', right_on='ORDER_DETAIL_CODE')
returned_orders = returned_orders[['PRODUCT_NUMBER', 'QUANTITY', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'RETAILER_SITE_CODE', 'ORDER_DATE', 'RETURN_DATE', 'RETURN_REASON_CODE']]
# returned_orders['RETURN_DATE'] = returned_orders['RETURN_DATE'].fillna(0)
# returned_orders['RETURN_REASON_CODE'] = returned_orders['RETURN_REASON_CODE'].fillna(0)
# returned_orders = returned_orders[returned_orders['RETURN_REASON_CODE'] != 0]
returned_orders['ORDER_DATE'] = pd.to_datetime(returned_orders['ORDER_DATE'])
returned_orders['RETURN_DATE'] = pd.to_datetime(returned_orders['RETURN_DATE'], format="%d-%m-%Y %H:%M:%S")
returned_orders['RETURN_DAY'] = returned_orders['RETURN_DATE'].dt.day
returned_orders['RETURN_MONTH'] = returned_orders['RETURN_DATE'].dt.month
returned_orders['RETURN_YEAR'] = returned_orders['RETURN_DATE'].dt.year
returned_orders = returned_orders.drop('RETURN_DATE', axis=1)
returned_orders['ORDER_DATE'] = pd.to_datetime(returned_orders['ORDER_DATE'])
returned_orders['ORDER_DAY'] = returned_orders['ORDER_DATE'].dt.day
returned_orders['ORDER_MONTH'] = returned_orders['ORDER_DATE'].dt.month
returned_orders['ORDER_YEAR'] = returned_orders['ORDER_DATE'].dt.year
returned_orders = returned_orders.drop('ORDER_DATE', axis=1)
returned_orders = returned_orders[returned_orders['RETURN_REASON_CODE'].notna()]
returned_orders

,PRODUCT_NUMBER,QUANTITY,UNIT_PRICE,UNIT_SALE_PRICE,RETAILER_SITE_CODE,RETURN_REASON_CODE,RETURN_DAY,RETURN_MONTH,RETURN_YEAR,ORDER_DAY,ORDER_MONTH,ORDER_YEAR
143,15,16,756.82,593.5,327,4,9.0,8.0,2021.0,26,7,2021
146,16,64,2.05,2.05,330,5,6.0,7.0,2021.0,28,6,2021
196,30,22,13.57,13.57,327,4,6.0,7.0,2021.0,25,6,2021
247,43,52,189.77,189.77,327,1,14.0,12.0,2021.0,23,10,2021
634,111,6,180.13,180.13,331,4,19.0,12.0,2021.0,3,12,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
42809,15,16,831.88,626.94,326,1,9.0,11.0,2020.0,16,10,2020
42816,17,34,90.09,90.09,329,1,18.0,8.0,2020.0,1,8,2020
42876,33,70,29.44,29.44,326,1,22.0,11.0,2020.0,27,9,2020
42879,34,54,51.24,51.24,329,3,17.0,10.0,2020.0,6,10,2020
